In [1]:
import pandas as pd

# ---- 路径：改成你的实际路径 ----
file_path = "/Users/muyuanchen/Desktop/Nat2018PublicUS.c20190509.r20190717.txt"

# =====================================================================
# 1-based（包含右端） → 0-based 半开区间 (start-1, end)
# 已含之前所有页到 ILLB_R，并修正 ILLB_R；再新增本页 5 个变量
# 同时修正：MRACE6 仅 1 位；MRACE31 列名拼写
# =====================================================================
colspecs = [
    # 出生基本信息
    (8, 12),    # DOB_YY  (9–12)
    (12, 14),   # DOB_MM  (13–14)
    (18, 22),   # DOB_TT  (19–22)
    (22, 23),   # DOB_WK  (23)
    (31, 32),   # BFACIL  (32)
    (49, 50),   # BFACIL3 (50)

    # 母亲年龄
    (72, 73),   # MAGE_IMPFLG (73)
    (73, 74),   # MAGE_REPFLG (74)
    (74, 76),   # MAGER      (75–76)
    (76, 78),   # MAGER14    (77–78)
    (78, 79),   # MAGER9     (79)

    # 母亲国籍/居住/种族
    (83, 84),   # MBSTATE_REC (84)
    (103,104),  # RESTATUS    (104)
    (104,106),  # MRACE31     (105–106)
    (106,107),  # MRACE6      (107)
    (108,110),  # MRACE15     (108–109)
    (109,110),  # MBRACE      (110)
    (110,111),  # MRACEIMP    (111)

    # 母亲族裔
    (111,112),  # MHISPX    (112)
    (114,115),  # MHISP_R   (115)
    (115,116),  # F_MHISP   (116)
    (116,117),  # MRACEHISP (117)

    # 婚姻状况 / 父系报告
    (118,119),  # MAR_P   (119)
    (119,120),  # DMAR    (120)
    (120,121),  # MAR_IMP (121)
    (122,123),  # F_MAR_P (123)

    # 教育（母/父）
    (123,124),  # MEDUC      (124)
    (125,126),  # F_MEDUC    (126)
    (141,142),  # FAGERPT_FLG (142)
    (146,148),  # FAGECOMB    (147–148)
    (148,150),  # FAGEREC11   (149–150)
    (150,152),  # FRACE31     (151–152)
    (152,153),  # FRACE6      (153)
    (153,155),  # FRACE15     (154–155)
    (158,159),  # FHISPX      (159)
    (159,160),  # FHISP_R     (160)
    (160,161),  # F_FHISP     (161)
    (161,162),  # FRACEHISP   (162)
    (162,163),  # FEDUC       (163)
    (164,165),  # F_FEDUC     (165)

    # 既往妊娠史
    (170,172),  # PRIORLIVE (171–172)
    (172,174),  # PRIORDEAD (173–174)
    (174,176),  # PRIORTERM (175–176)
    (178,179),  # LBO_REC   (179)
    (181,182),  # TBO_REC   (182)

    # 间隔 Recode
    (197,200),  # ILLB_R   (198–200)
    (200,202),  # ILLB_R11 (201–202)
    (205,208),  # ILOP_R   (206–208)
    (208,210),  # ILOP_R11 (209–210)
    (213,216),  # ILP_R    (214–216)
    (216,218),  # ILP_R11  (217–218)

    # 产前检查
    (223,225),  # PRECARE    (224–225)
    (225,226),  # F_MPCB     (226)
    (226,227),  # PRECARE5   (227)
    (237,239),  # PREVIS     (238–239)
    (241,243),  # PREVIS_REC (242–243)
    (243,244),  # F_TPCV     (244)

    # WIC & 吸烟
    (250,251),  # WIC     (251)
    (251,252),  # F_WIC   (252)
    (252,254),  # CIG_0   (253–254)
    (254,256),  # CIG_1   (255–256)
    (256,258),  # CIG_2   (257–258)
    (258,260),  # CIG_3   (259–260)
    (260,261),  # CIG0_R  (261)
    (261,262),  # CIG1_R  (262)
    (262,263),  # CIG2_R  (263)
    (263,264),  # CIG3_R  (264)
    (264,265),  # F_CIGS_0 (265)
    (265,266),  # F_CIGS_1 (266)
    (266,267),  # F_CIGS_2 (267)
    (267,268),  # F_CIGS_3 (268)
    (268,269),  # CIG_REC  (269)
    (269,270),  # F_TOBACO (270)

    # 母亲身高 / BMI
    (279,281),  # M_HT_IN (280–281)
    (281,282),  # F_M_HT  (282)
    (282,286),  # BMI     (283–286)
    (286,287),  # BMI_R     (287) Body Mass Index Recode
    # 288–291 FILLER 跳过
    (291,294),  # PWGT_R    (292–294) Pre-pregnancy Weight Recode
    (294,295),  # F_PWGT    (295) Reporting Flag for Pre-pregnancy Weight
    # 296–298 FILLER 跳过
    (298,301),  # DWGT_R    (299–301) Delivery Weight Recode
    # 302 FILLER 跳过
    (302,303),  # F_DWGT    (303) Reporting Flag for Delivery Weight
    (303,305),  # WTGAIN    (304–305) Weight Gain
    (305,306),  # WTGAIN_REC (306) Weight Gain Recode
    (306,307),  # F_WTGAIN  (307) Reporting Flag for Weight Gain
    # 308–312 FILLER 跳过
    (312,313),  # RF_PDIAB   (313) Pre-pregnancy Diabetes
    (313,314),  # RF_GDIAB   (314) Gestational Diabetes
    (314,315),  # RF_PHYPE   (315) Pre-pregnancy Hypertension
    (315,316),  # RF_GHYPE   (316) Gestational Hypertension
    (316,317),  # RF_EHYPE   (317) Hypertension Eclampsia
    (317,318),  # RF_PPTTERM (318) Previous Preterm Birth
    (318,319),  # F_RF_PDIAB (319) Reporting Flag for Pre-pregnancy Diabetes
    (319,320),  # F_RF_GDIAB (320) Reporting Flag for Gestational Diabetes
    (320,321),  # F_RF_PHYPER (321) Reporting Flag for Pre-pregnancy Hypertension
    (321,322),  # F_RF_GHYPER (322) Reporting Flag for Gestational Hypertension
    (322,323),  # F_RF_ECLAMP (323) Reporting Flag for Hypertension Eclampsia
    (323,324),  # F_RF_PPB   (324) Reporting Flag for Previous Preterm Birth
    (324,325),  # RF_INFTR   (325) Infertility Treatment Used
    (325,326),  # RF_FEDRG    (326) Fertility Enhancing Drugs
    (326,327),  # RF_ARTEC    (327) Assisted Reproductive Technology
    (327,328),  # F_RF_INFT   (328) Reporting Flag for Infertility Treatment
    (328,329),  # F_RF_INF_DRG (329) Reporting Flag for Fertility Enhance Drugs
    (329,330),  # F_RF_INF_ART (330) Reporting Flag for Reproductive Technology
    (330,331),  # RF_CESAR    (331) Previous Cesarean
    (331,333),  # RF_CESARN   (332–333) Number of Previous Cesareans
    # 334 filler 跳过
    (334,335),  # F_RF_CESAR  (335) Reporting Flag for Previous Cesarean
    (335,336),  # F_RF_NCESAR (336) Reporting Flag for Number of Previous Cesareans
    (336,337),
    (342,343),  # IP_GON     (343) Gonorrhea
    (343,344),  # IP_SYPH    (344) Syphilis
    (344,345),  # IP_CHLAM   (345) Chlamydia
    (345,346),  # IP_HEPB    (346) Hepatitis B
    (346,347),  # IP_HEPC    (347) Hepatitis C
    (347,348),  # F_IP_GONOR (348) Reporting Flag for Gonorrhea
    (348,349),  # F_IP_SYPH  (349) Reporting Flag for Syphilis
    (349,350),  # F_IP_CHLAM (350) Reporting Flag for Chlamydia
    (350,351),  # F_IP_HEPATB (351) Reporting Flag for Hepatitis B
    (351,352),  # F_IP_HEPATC (352) Reporting Flag for Hepatitis C
    (352,353),  # NO_INFEC   (353) No Infections Reported
    # 354–358 filler 跳过
    (359,360),  # OB_ECVS   (360) Successful External Cephalic Version
    (360,361),  # OB_ECVF   (361) Failed External Cephalic Version
    # 362 filler 跳过
    (362,363),  # F_OB_SUCC (363) Reporting Flag for Successful External Cephalic Version
    (363,364),  # F_OB_FAIL (364) Reporting Flag for Failed External Cephalic Version
    # 365–382 filler 跳过
    (382,383),  # LD_INDL   (383) Induction of Labor
    (383,384),  # LD_AUGM   (384) Augmentation of Labor
    (384,385),  # LD_STER   (385) Steroids
    (385,386),  # LD_ANTB   (386) Antibiotics
    (386,387),  # LD_CHOR   (387) Chorioamnionitis
    (387,388),  # LD_ANES    (388) Anesthesia
    (388,389),  # F_LD_INDL  (389) Reporting Flag for Induction of Labor
    (389,390),  # F_LD_AUGM  (390) Reporting Flag for Augmentation of Labor
    (390,391),  # F_LD_STER  (391) Reporting Flag for Steroids
    (391,392),  # F_LD_ANTB  (392) Reporting Flag for Antibiotics
    (392,393),  # F_LD_CHOR  (393) Reporting Flag for Chorioamnionitis
    (393,394),  # F_LD_ANES  (394) Reporting Flag for Anesthesia
    (394,395),  # NO_LBRDLV  (395) No Characteristics of Labor Reported
    # 396–400 filler 跳过
    (400,401),  # ME_PRES    (401) Fetal Presentation at Delivery
    (401,402),  # ME_ROUT    (402) Final Route & Method of Delivery
    (402,403),  # ME_TRIAL   (403) Trial of Labor Attempted (if cesarean)
    (403,404),  # F_ME_PRES    (404) Reporting Flag for Fetal Presentation
    (404,405),  # F_ME_ROUT    (405) Reporting Flag for Final Route & Method of Delivery
    (405,406),  # F_ME_TRIAL   (406) Reporting Flag for Trial of Labor Attempted
    (406,407),  # RDMETH_REC   (407) Delivery Method Recode (detailed)
    (407,408),  # DMETH_REC    (408) Delivery Method Recode (simple: vaginal / C-section / unknown)
    (408,409),  # F_DMETH_REC  (409) Reporting Flag for Method of Delivery Recode
    # 410–414 filler 跳过
    (414,415),  # MM_MTR       (415) Maternal Transfusion
    (415,416),  # MM_PLAC      (416) Perineal Laceration
    (416,417),  # MM_RUPT      (417) Ruptured Uterus
    (417,418),  # MM_UHYST     (418) Unplanned Hysterectomy
    (418,419),  # MM_AICU    (419) Admit to Intensive Care
    # 420 filler 跳过
    (420,421),  # F_MM_MTR   (421) Reporting Flag for Maternal Transfusion
    (421,422),  # F_MM_PLAC  (422) Reporting Flag for Perineal Laceration
    (422,423),  # F_MM_RUPT  (423) Reporting Flag for Ruptured Uterus
    (423,424),  # F_MM_UHYST (424) Reporting Flag for Unplanned Hysterectomy
    (424,425),  # F_MM_AICU  (425) Reporting Flag for Admission to Intensive Care
    # 426 filler 跳过
    (426,427),  # NO_MMORB   (427) No Maternal Morbidity Reported
    # 428–432 filler 跳过
    (432,433),  # ATTEND     (433) Attendant at Birth
    (433,434),  # MITRAN     (434) Mother Transferred
    (434,435),  # PAY        (435) Payment Source for Delivery
    # === 本页继续（436–450）: Payment Recode + APGAR ===
    (435,436),  # PAY_REC   (436) Payment Recode
    (436,437),  # F_PAY     (437) Reporting Flag for Source of Payment
    (437,438),  # F_PAY_REC (438) Reporting Flag for Payment Recode
    # 439–443 filler 跳过
    (443,445),  # APGAR5    (444–445) Five Minute APGAR Score
    (445,446),  # APGAR5R   (446) Five Minute APGAR Recode
    (446,447),  # F_APGAR5  (447) Reporting Flag for Five Minute APGAR
    (447,449),  # APGAR10   (448–449) Ten Minute APGAR Score
    (449,450),  # APGAR10R  (450) Ten Minute APGAR Recode
    # 451–453 filler 跳过
    (453,454),  # DPLURAL   (454) Plurality Recode
    # 455 filler 跳过
    (455,456),  # IMP_PLUR  (456) Plurality Imputed
    # 457–458 filler 跳过
    (458,459),  # SETORDER_R (459) Set Order Recode
    # 460–474 filler 跳过
    (474,475),  # SEX       (475) Sex of Infant
    (475,476),  # IMP_SEX   (476) Imputed Sex
    (476,478),  # DLMP_MM   (477–478) Last Normal Menses Month
    # 479–480 filler 跳过
    (480,484),  # DLMP_YY    (481–484) Last Normal Menses Year
    # 485–487 filler 跳过
    (487,488),  # COMPGST_IMP (488) Combined Gestation Imputation Flag
    (488,489),  # OBGEST_FLG  (489) Obstetric Estimate of Gestation Used Flag
    (489,491),  # COMBGEST    (490–491) Combined Gestation – Detail in Weeks
    (491,493),  # GESTREC10   (492–493) Combined Gestation Recode 10
    (493,494),  # GESTREC3    (494) Combined Gestation Recode 3
    # 495–497 filler 跳过
    (497,498),  # LMPUSED     (498) Combined Gestation Used Flag
    (498,500),  # OEGest_Comb (499–500) Obstetric Estimate Edited (weeks)
    (500,502),  # OEGest_R10  (501–502) Obstetric Estimate Recode 10
    # === 本页继续（503–518）: Birth Weight + Abnormal Newborn Conditions ===
    (502,503),   # OEGest_R3 (503) Obstetric Estimate Recode 3
    (503,507),   # DBWT      (504–507) Birth Weight – Detail in Grams (Edited)
    # 508 filler 跳过
    (508,510),   # BWTR12    (509–510) Birth Weight Recode 12
    (510,511),   # BWTR4     (511) Birth Weight Recode 4
    # 512–516 filler 跳过
    (516,517),   # AB_AVEN1  (517) Assisted Ventilation (immediate)
    (517,518),   # AB_AVEN6  (518) Assisted Ventilation > 6 hrs
    # === 本页继续（519–531）: Abnormal Conditions of the Newborn ===
    (518,519),  # AB_NICU    (519) Admission to NICU
    (519,520),  # AB_SURF    (520) Surfactant
    (520,521),  # AB_ANTI    (521) Antibiotics for Newborn
    (521,522),  # AB_SEIZ    (522) Seizures
    # 523 filler 跳过
    (523,524),  # F_AB_VENT   (524) Reporting Flag for Assisted Ventilation (immediate)
    (524,525),  # F_AB_VENT6  (525) Reporting Flag for Assisted Ventilation >6 hrs
    (525,526),  # F_AB_NIUC   (526) Reporting Flag for Admission to NICU
    (526,527),  # F_AB_SURFAC (527) Reporting Flag for Surfactant
    (527,528),  # F_AB_ANTIBIO (528) Reporting Flag for Antibiotics
    (528,529),  # F_AB_SEIZ   (529) Reporting Flag for Seizures
    # 530 filler 跳过
    (530,531),  # NO_ABNORM   (531) No Abnormal Conditions Checked
    # 532–536 filler 跳过
    (536,537),  # CA_ANEN   (537) Anencephaly
    # === 本页继续（537–548）: Congenital Anomalies of the Newborn ===
    (537,538),  # CA_MNSB   (538) Meningomyelocele / Spina Bifida
    (538,539),  # CA_CCHD   (539) Cyanotic Congenital Heart Disease
    (539,540),  # CA_CDH    (540) Congenital Diaphragmatic Hernia
    (540,541),  # CA_OMPH   (541) Omphalocele
    (541,542),  # CA_GAST   (542) Gastroschisis
    (542,543),  # F_CA_ANEN  (543) Reporting Flag for Anencephaly
    (543,544),  # F_CA_MENIN (544) Reporting Flag for Meningomyelocele/Spina Bifida
    (544,545),  # F_CA_HEART (545) Reporting Flag for Congenital Heart Disease
    (545,546),  # F_CA_HERNIA (546) Reporting Flag for Congenital Diaphragmatic Hernia
    (546,547),  # F_CA_OMPHA  (547) Reporting Flag for Omphalocele
    (547,548),  # F_CA_GASTRO (548) Reporting Flag for Gastroschisis
    # === 本页继续（549–561）: Congenital Anomalies (continued) ===
    (548,549),  # CA_LIMB    (549) Limb Reduction Defect
    (549,550),  # CA_CLEFT   (550) Cleft Lip with/without Cleft Palate
    (550,551),  # CA_CLPAL   (551) Cleft Palate alone
    (551,552),  # CA_DOWN    (552) Down Syndrome
    (552,553),  # CA_DISOR   (553) Suspected Chromosomal Disorder
    (553,554),  # CA_HYPO    (554) Hypospadias
    (554,555),  # F_CA_LIMB   (555) Reporting Flag for Limb Reduction Defect
    (555,556),  # F_CA_CLEFTLP (556) Reporting Flag for Cleft Lip
    (556,557),  # F_CA_CLEFT   (557) Reporting Flag for Cleft Palate
    (557,558),  # F_CA_DOWNS   (558) Reporting Flag for Down Syndrome
    (558,559),  # F_CA_CHROM   (559) Reporting Flag for Suspected Chromosomal Disorder
    (559,560),  # F_CA_HYPOS   (560) Reporting Flag for Hypospadias
    (560,561),  # NO_CONGEN    (561) No Congenital Anomalies Checked
    # === 本页继续（567–570）: Infant Outcomes ===
    (566,567),  # ITRAN   (567) Infant Transferred
    (567,568),  # ILIVE   (568) Infant Living at Time of Report
    (568,569),  # BFED    (569) Infant Breastfed at Discharge
    (569,570),  # F_BFED  (570) Reporting Flag for Breastfed at Discharge
    # 571–1330 filler 跳过


]



colnames = [
    "DOB_YY","DOB_MM","DOB_TT","DOB_WK","BFACIL","BFACIL3",
    "MAGE_IMPFLG","MAGE_REPFLG","MAGER","MAGER14","MAGER9",
    "MBSTATE_REC","RESTATUS","MRACE31","MRACE6","MRACE15","MBRACE","MRACEIMP",
    "MHISPX","MHISP_R","F_MHISP","MRACEHISP",
    "MAR_P","DMAR","MAR_IMP","F_MAR_P",
    "MEDUC","F_MEDUC","FAGERPT_FLG","FAGECOMB","FAGEREC11","FRACE31","FRACE6","FRACE15",
    "FHISPX","FHISP_R","F_FHISP","FRACEHISP","FEDUC","F_FEDUC",
    "PRIORLIVE","PRIORDEAD","PRIORTERM","LBO_REC","TBO_REC",
    "ILLB_R","ILLB_R11","ILOP_R","ILOP_R11","ILP_R","ILP_R11",
    "PRECARE","F_MPCB","PRECARE5","PREVIS","PREVIS_REC","F_TPCV",
    "WIC","F_WIC","CIG_0","CIG_1","CIG_2","CIG_3",
    "CIG0_R","CIG1_R","CIG2_R","CIG3_R",
    "F_CIGS_0","F_CIGS_1","F_CIGS_2","F_CIGS_3",
    "CIG_REC","F_TOBACO",
    "M_HT_IN","F_M_HT","BMI",
    "BMI_R","PWGT_R","F_PWGT","DWGT_R","F_DWGT","WTGAIN","WTGAIN_REC","F_WTGAIN",
    "RF_PDIAB","RF_GDIAB","RF_PHYPE","RF_GHYPE","RF_EHYPE","RF_PPTTERM",
    "F_RF_PDIAB","F_RF_GDIAB","F_RF_PHYPER","F_RF_GHYPER","F_RF_ECLAMP","F_RF_PPB",
    "RF_INFTR",
    "RF_FEDRG","RF_ARTEC","F_RF_INFT","F_RF_INF_DRG","F_RF_INF_ART",
    "RF_CESAR","RF_CESARN","F_RF_CESAR","F_RF_NCESAR","NO_RISKS",
    "IP_GON","IP_SYPH","IP_CHLAM","IP_HEPB","IP_HEPC",
    "F_IP_GONOR","F_IP_SYPH","F_IP_CHLAM","F_IP_HEPATB","F_IP_HEPATC",
    "NO_INFEC",
    "OB_ECVS","OB_ECVF","F_OB_SUCC","F_OB_FAIL",
    "LD_INDL","LD_AUGM","LD_STER","LD_ANTB","LD_CHOR",
    "LD_ANES",
    "F_LD_INDL","F_LD_AUGM","F_LD_STER","F_LD_ANTB","F_LD_CHOR","F_LD_ANES",
    "NO_LBRDLV",
    "ME_PRES","ME_ROUT","ME_TRIAL",
    "F_ME_PRES","F_ME_ROUT","F_ME_TRIAL",
    "RDMETH_REC","DMETH_REC","F_DMETH_REC",
    "MM_MTR","MM_PLAC","MM_RUPT","MM_UHYST",
    "MM_AICU",
    "F_MM_MTR","F_MM_PLAC","F_MM_RUPT","F_MM_UHYST","F_MM_AICU",
    "NO_MMORB",
    "ATTEND","MITRAN","PAY",
    "PAY_REC","F_PAY","F_PAY_REC",
    "APGAR5","APGAR5R","F_APGAR5","APGAR10","APGAR10R",
    "DPLURAL","IMP_PLUR","SETORDER_R",
    "SEX","IMP_SEX","DLMP_MM",
    "DLMP_YY","COMPGST_IMP","OBGEST_FLG","COMBGEST","GESTREC10","GESTREC3",
    "LMPUSED","OEGest_Comb","OEGest_R10",
    "OEGest_R3","DBWT","BWTR12","BWTR4","AB_AVEN1","AB_AVEN6",
    "AB_NICU","AB_SURF","AB_ANTI","AB_SEIZ",
    "F_AB_VENT","F_AB_VENT6","F_AB_NIUC","F_AB_SURFAC","F_AB_ANTIBIO","F_AB_SEIZ",
    "NO_ABNORM",
    "CA_ANEN","CA_MNSB","CA_CCHD","CA_CDH","CA_OMPH","CA_GAST",
    "F_CA_ANEN","F_CA_MENIN","F_CA_HEART","F_CA_HERNIA","F_CA_OMPHA","F_CA_GASTRO",
    "CA_LIMB","CA_CLEFT","CA_CLPAL","CA_DOWN","CA_DISOR","CA_HYPO",
    "F_CA_LIMB","F_CA_CLEFTLP","F_CA_CLEFT","F_CA_DOWNS","F_CA_CHROM","F_CA_HYPOS",
    "NO_CONGEN",
    "ITRAN","ILIVE","BFED","F_BFED"
]

# ---- 读取前 10000 行，先按字符串读，避免空白自动变 NaN ----
df_raw = pd.read_fwf(
    file_path, colspecs=colspecs, names=colnames,
    nrows=10000, dtype="string", keep_default_na=False, na_filter=False
)

# 清理空格并尽量转数值（保留真正的空为 NaN）
df = df_raw.apply(lambda s: s.str.strip())
for c in colnames:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print(df.head(10))



   DOB_YY  DOB_MM  DOB_TT  DOB_WK  BFACIL  BFACIL3  MAGE_IMPFLG  MAGE_REPFLG  \
0    2018       1    1227       2       1        1         <NA>         <NA>   
1    2018       1    1704       2       1        1         <NA>         <NA>   
2    2018       1     336       2       1        1         <NA>         <NA>   
3    2018       1     938       2       1        1         <NA>         <NA>   
4    2018       1     830       3       1        1         <NA>         <NA>   
5    2018       1      28       2       2        2         <NA>         <NA>   
6    2018       1     341       3       1        1         <NA>         <NA>   
7    2018       1    1615       4       1        1         <NA>         <NA>   
8    2018       1    1754       5       1        1         <NA>         <NA>   
9    2018       1    1111       6       1        1         <NA>         <NA>   

   MAGER  MAGER14  ...  F_CA_CLEFTLP  F_CA_CLEFT  F_CA_DOWNS  F_CA_CHROM  \
0     30       10  ...             1       

In [2]:
import pandas as pd

# ---- 假设你前面已经 parse 完成，得到 df ----
# 这里 df 就是 pd.read_fwf(...) 读进来的 DataFrame

# 删除所有全是 NaN 或空白的列
import numpy as np

# 把 <NA> 统一转成 np.nan
import numpy as np

df_clean = df.copy()
# 统一把 <NA> 转成 np.nan
df_clean = df_clean.replace({pd.NA: np.nan})

# 去掉全是 nan/空格/空字符串的列
df_clean = df_clean.loc[:, ~df_clean.replace(r'^\s*$', np.nan, regex=True).isna().all(axis=0)]


print("原始列数:", df.shape[1])
print("清理后列数:", df_clean.shape[1])


# 保存一下结果
df_clean.to_csv("nat2018_clean.csv", index=False)

print(df_clean.head(10))


原始列数: 227
清理后列数: 168
   DOB_YY  DOB_MM  DOB_TT  DOB_WK  BFACIL  BFACIL3  MAGER  MAGER14  MAGER9  \
0    2018       1    1227       2       1        1     30       10       5   
1    2018       1    1704       2       1        1     35       11       6   
2    2018       1     336       2       1        1     28        9       4   
3    2018       1     938       2       1        1     23        8       3   
4    2018       1     830       3       1        1     37       11       6   
5    2018       1      28       2       2        2     26        9       4   
6    2018       1     341       3       1        1     28        9       4   
7    2018       1    1615       4       1        1     31       10       5   
8    2018       1    1754       5       1        1     37       11       6   
9    2018       1    1111       6       1        1     26        9       4   

   MBSTATE_REC  ...  F_CA_OMPHA  F_CA_GASTRO  F_CA_LIMB  F_CA_CLEFTLP  \
0            1  ...           1            1   

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
import sklearn

# ========= 0) 数据准备 =========
df = df_clean.copy()
df.columns = df.columns.str.lower()
assert "dbwt" in df.columns, "找不到 dbwt 列（出生体重）"

# 丢掉 y 缺失 & 明显异常
df = df[df["dbwt"].notna()].copy()
df = df[(df["dbwt"] >= 500) & (df["dbwt"] <= 6000)].copy()

# 强制移除派生泄漏列
leak_cols = ["bwtr12", "bwtr4"]
df = df.drop(columns=[c for c in leak_cols if c in df.columns])

# 特征/目标
X = df.drop(columns=["dbwt"])
y = df["dbwt"].astype(float).values

# 列类型划分
num_cols = X.select_dtypes(include="number").columns.tolist()
cat_cols = X.select_dtypes(exclude="number").columns.tolist()
print(f"[INFO] 使用数值列 {len(num_cols)}，类别列 {len(cat_cols)}")

# ========= 1) 先切分，再在训练集上拟合（避免泄漏）=========
Xtr_raw, Xva_raw, ytr, yva = train_test_split(X, y, test_size=0.2, random_state=42)

# ========= 2) 预处理（数值：中位数填充+标准化；类别：众数填充+OneHot(稠密)）=========
# sklearn 1.2+ 用 sparse_output；老版本用 sparse=False
ohe_kwargs = {}
ver = tuple(map(int, sklearn.__version__.split(".")[:2]))
if ver >= (1, 2):
    ohe_kwargs["sparse_output"] = False
else:
    ohe_kwargs["sparse"] = False

num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", **ohe_kwargs))
])

preprocess = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

# ========= 3) PCA + Linear Regression ==========
model = Pipeline([
    ("preprocess", preprocess),
    ("pca", PCA(n_components=50, svd_solver="full", random_state=42)),  # 或 n_components=0.95
    ("linreg", LinearRegression())
])

# ========= 4) 训练与评估 ==========
model.fit(Xtr_raw, ytr)
pred = model.predict(Xva_raw)

rmse = root_mean_squared_error(yva, pred)
r2 = r2_score(yva, pred)

pca = model.named_steps["pca"]
expl = pca.explained_variance_ratio_.sum()
print(f"[INFO] PCA 主成分数: {pca.n_components_}, 累计解释方差: {expl:.3f}")
print(f"[RESULT] PCA+Linear Regression → RMSE: {rmse:.1f} g | R²: {r2:.3f}")
print(f"[SANITY] y(mean±std)={ytr.mean():.1f}±{ytr.std():.1f}, pred(min,max)=({pred.min():.1f}, {pred.max():.1f})")


[INFO] 使用数值列 165，类别列 0
[INFO] PCA 主成分数: 50, 累计解释方差: 0.980
[RESULT] PCA+Linear Regression → RMSE: 469.0 g | R²: 0.322
[SANITY] y(mean±std)=3401.1±584.4, pred(min,max)=(1665.6, 6339.8)


In [10]:
# =========================================================
# 0) Imports & 全局设置
# =========================================================
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts

np.random.seed(42)
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] device =", device, "| sklearn:", sklearn.__version__, "| torch:", torch.__version__)

# =========================================================
# 1) 数据准备（以 df_clean 为源）
# =========================================================
df = df_clean.copy()
df.columns = df.columns.str.lower()
assert "dbwt" in df.columns, "找不到 dbwt 列（出生体重，克）"

# 丢掉 y 缺失与异常
df = df[df["dbwt"].notna()].copy()
df = df[(df["dbwt"] >= 500) & (df["dbwt"] <= 6000)].copy()

# 移除潜在泄漏列（如果存在）
leak_cols = ["bwtr12", "bwtr4"]
df = df.drop(columns=[c for c in leak_cols if c in df.columns])

# 特征与目标
X = df.drop(columns=["dbwt"])
y = df["dbwt"].astype(float).values

# 列类型划分
num_cols = X.select_dtypes(include="number").columns.tolist()
cat_cols = X.select_dtypes(exclude="number").columns.tolist()
print(f"[INFO] 使用数值列 {len(num_cols)}，类别列 {len(cat_cols)}")

# =========================================================
# 2) 三划分：train / val / test（回归用分箱伪分层）
# =========================================================
def make_strata(y, q=10):
    try:
        return pd.qcut(y, q=q, labels=False, duplicates="drop")
    except Exception:
        return None

strata = make_strata(y, q=10)

# 70% train, 15% val, 15% test
Xtr_raw, Xtemp, ytr, ytemp, strata_tr, strata_temp = train_test_split(
    X, y, strata, test_size=0.30, random_state=42,
    stratify=strata if strata is not None else None
)

Xva_raw, Xte_raw, yva, yte, strata_va, strata_te = train_test_split(
    Xtemp, ytemp, strata_temp, test_size=0.50, random_state=42,
    stratify=strata_temp if strata is not None else None
)

print(f"[SPLIT] train={len(ytr)} | val={len(yva)} | test={len(yte)}")

# =========================================================
# 3) 预处理（数值：中位数填充+标准化；类别：众数填充+OneHot）
# =========================================================
ohe_kwargs = {}
ver = tuple(map(int, sklearn.__version__.split(".")[:2]))
if ver >= (1, 2):
    ohe_kwargs["sparse_output"] = False
else:
    ohe_kwargs["sparse"] = False

num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", **ohe_kwargs))
])

preprocess = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

# =========================================================
# 4) PCA（只在训练集上 fit），n_components=150（可调或用 0.95）
# =========================================================
pca_dim_target = 50  # 你指定的维度；也可改为 0.95（保留95%方差）
prep_pca = Pipeline([
    ("preprocess", preprocess),
    ("pca", PCA(n_components=pca_dim_target, svd_solver="full", random_state=42))
])

prep_pca.fit(Xtr_raw, ytr)
Ztr = prep_pca.transform(Xtr_raw)
Zva = prep_pca.transform(Xva_raw)
Zte = prep_pca.transform(Xte_raw)

pca = prep_pca.named_steps["pca"]
expl = pca.explained_variance_ratio_.sum()
print(f"[INFO] PCA 主成分数: {pca.n_components_} | 累计解释方差: {expl:.3f}")

# =========================================================
# 5) 目标标准化（提升回归稳定性）
# =========================================================
y_mean, y_std = ytr.mean(), ytr.std()
ytr_std = (ytr - y_mean) / y_std
yva_std = (yva - y_mean) / y_std
yte_std = (yte - y_mean) / y_std

# =========================================================
# 6) 张量与 DataLoader
# =========================================================
Xtr_t = torch.tensor(Ztr, dtype=torch.float32)
Xva_t = torch.tensor(Zva, dtype=torch.float32)
Xte_t = torch.tensor(Zte, dtype=torch.float32)

ytr_t = torch.tensor(ytr_std, dtype=torch.float32).unsqueeze(1)
yva_t = torch.tensor(yva_std, dtype=torch.float32).unsqueeze(1)
yte_t = torch.tensor(yte_std, dtype=torch.float32).unsqueeze(1)

train_loader = DataLoader(TensorDataset(Xtr_t, ytr_t), batch_size=256, shuffle=True)
val_loader   = DataLoader(TensorDataset(Xva_t, yva_t), batch_size=1024, shuffle=False)
test_loader  = DataLoader(TensorDataset(Xte_t, yte_t), batch_size=1024, shuffle=False)

# =========================================================
# 7) 模型（两种二选一）——默认启用「简单版」
# =========================================================
USE_STRONG_RESIDUAL = False  # ← 想用更强的残差版就改为 True

class PreNormResidual(nn.Module):
    def __init__(self, dim, hidden, dropout=0.3):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.ff1 = nn.Sequential(nn.Linear(dim, hidden), nn.SiLU(), nn.Dropout(dropout))
        self.norm2 = nn.LayerNorm(hidden)
        self.ff2 = nn.Linear(hidden, dim)
    def forward(self, x):
        h = self.ff1(self.norm1(x))
        h = self.ff2(self.norm2(h))
        return x + h

class StrongRegressor(nn.Module):
    def __init__(self, in_dim, width=256, depth=4, dropout=0.3):
        super().__init__()
        self.inp = nn.Sequential(nn.LayerNorm(in_dim), nn.Linear(in_dim, width), nn.SiLU())
        self.blocks = nn.Sequential(*[PreNormResidual(width, width, dropout) for _ in range(depth)])
        self.head = nn.Sequential(nn.LayerNorm(width), nn.Linear(width, width//2), nn.SiLU(), nn.Linear(width//2, 1))
    def forward(self, x):
        x = self.inp(x); x = self.blocks(x); return self.head(x)

model = StrongRegressor(pca_dim_target).to(device)


in_dim = Ztr.shape[1]
if USE_STRONG_RESIDUAL:
    model = StrongRegressor(in_dim=in_dim, width=256, depth=4, dropout=0.3).to(device)
    criterion = nn.MSELoss()
    optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=2e-5)
    scheduler_mode = "cosine"
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2)
else:
    model = PcaRegressor(in_dim).to(device)
    # 注意：你之前设置 lr=1e-1 偏大，容易不稳；这里用 1e-3 更靠谱
    criterion = nn.MSELoss()                          # 更贴近 RMSE 
    optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=2e-5)

    # 调度器：余弦退火 + warm restarts（代替 ReduceLROnPlateau）
    from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2)


print(model)

# =========================================================
# 8) 训练循环（带 EarlyStopping）
# =========================================================
epochs = 300
patience = 20
best_val = float("inf")
best_state = None
wait = 0

def evaluate_avg_loss(loader):
    model.eval()
    loss_sum, n = 0.0, 0
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = criterion(pred, yb)
            bs = xb.shape[0]
            loss_sum += loss.item() * bs
            n += bs
    return loss_sum / max(n, 1)

for epoch in range(1, epochs + 1):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

    val_loss = evaluate_avg_loss(val_loader)

    # 调度
    if scheduler_mode == "plateau":
        scheduler.step(val_loss)
    else:
        scheduler.step(epoch)

    # 早停
    if val_loss < best_val - 1e-6:
        best_val = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        wait = 0
    else:
        wait += 1

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | val_loss={val_loss:.5f} (std-y 尺度) | lr={optimizer.param_groups[0]['lr']:.6f}")

    if wait >= patience:
        print(f"[EarlyStopping] 第 {epoch} 轮触发 | 最佳验证损失={best_val:.5f}")
        break

if best_state is not None:
    model.load_state_dict(best_state)

# =========================================================
# 9) 评估：验证集与测试集（还原到克单位）
# =========================================================
def inv_std(y_std_arr):
    return y_std_arr * y_std + y_mean

def eval_rmse_r2(X_t, y_true, name="SPLIT"):
    model.eval()
    with torch.no_grad():
        y_pred_std = model(X_t.to(device)).cpu().numpy().ravel()
    y_pred = inv_std(y_pred_std)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2   = r2_score(y_true, y_pred)
    print(f"[{name}] RMSE: {rmse:.1f} g | R²: {r2:.3f} | pred(min,max)=({y_pred.min():.1f},{y_pred.max():.1f})")
    return rmse, r2

_ = eval_rmse_r2(Xva_t, yva, name="VAL")
_ = eval_rmse_r2(Xte_t, yte, name="TEST")

# =========================================================
# 10) 推理函数（新样本 DataFrame → 预测克重）
# =========================================================
def predict_dbwt_with_torch(df_new: pd.DataFrame) -> np.ndarray:
    """
    输入：原始特征表 df_new（列需与训练时一致）
    输出：预测出生体重（克）
    """
    Z_new = prep_pca.transform(df_new)  # 预处理 + PCA
    X_new = torch.tensor(Z_new, dtype=torch.float32).to(device)
    model.eval()
    with torch.no_grad():
        y_pred_std = model(X_new).cpu().numpy().ravel()
    return inv_std(y_pred_std)


[INFO] device = cpu | sklearn: 1.4.2 | torch: 2.3.1
[INFO] 使用数值列 165，类别列 0
[SPLIT] train=6993 | val=1498 | test=1499
[INFO] PCA 主成分数: 50 | 累计解释方差: 0.980
PcaRegressor(
  (net): Sequential(
    (0): Linear(in_features=50, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): ReLU()
    (10): Linear(in_features=32, out_features=1, bias=True)
  )
)
Epoch 001 | val_loss=0.60809 (std-y 尺度) | lr=0.000998
Epoch 010 | val_loss=0.51391 (std-y 尺度) | lr=0.000998
Epoch 020 | val_loss=0.51192 (std-y 尺度) | lr=0.000998
Epoch 030 | val_loss=0.52739 (std-y 尺度) | lr=0.000998
Epoch 040 | val_loss=0.53524 (std-y 尺度) | l

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
